In [1]:
import os
import glob
import json
import tqdm
import zipfile
import io, json, re, zipfile, pickle, html
from pathlib import Path
from typing import Any, Dict, List, Optional
import pandas as pd
from config.settings import Settings 
from utils import *
settings = Settings()

# Extract content geel into dataframe

In [13]:
extracted = {}
json_docs = get_all_json_docs(content_type="geel")
for key, doc in json_docs.items():
    extracted_data = extract_json(key,doc)
    extracted[key] = extracted_data
extracted_df = pd.DataFrame(extracted.values())
print(f"Extracted {len(extracted_df)} documents")

Extracted 6146 documents


In [14]:
extracted_df_nodupes = extracted_df.drop_duplicates(subset='title')
print(f"Removed {len(extracted_df) - len(extracted_df_nodupes)} duplicates")

Removed 773 duplicates


### Koppelling met KM nummer

In [18]:
km_title_map = pd.read_excel(settings.content_folder / 'titel_km_map.xlsx').rename(columns={'Content Title':'title',"Article Identifier": 'km_nummer'})
km_title_map = km_title_map.drop_duplicates('title').set_index('title')
extracted_df_km= extracted_df_nodupes.set_index('title').join(km_title_map).dropna(subset='km_nummer')

In [21]:
extracted_df_km.to_pickle(settings.content_folder / "extracted_df_km.pkl")

### Lijst met duplicaten exporteren

In [10]:
extracted_df.set_index("id").loc[extracted_df.set_index("id").index.difference(extracted_df_nodupes.set_index("id").index)].to_excel("dupelist.xlsx")